#Colocando dados no sql e criando .dump

In [ ]:
#Crio a database e a conexão
conn = sqlite3.connect('test_database')
c = conn.cursor()

#Crio o esquema para a tabela (Incompleto, faltam chaves)
c.execute('CREATE TABLE IF NOT EXISTS especies (id_expecie number, nome_cientifico text, idPan text, pan_nome text)')
conn.commit()

#Passo as informações do dataframe pandas para a tabela sql
df_especies.to_sql('especies', conn, if_exists='replace', index = False)

#Crio o dump em sql (Não é de fato necessário no trabalho, mas é bom saber fazer)
with open('pandados.sql', 'w') as f:
    for line in conn.iterdump():
        f.write('%s\n' % line)
conn.close()

# Formatação das Tabelas - DEFINITIVO

## Importando bibliotecas

In [ ]:
import pandas as pd
import sqlite3
import io

## Abertura das tabelas

In [ ]:
pan_csv = pd.read_csv('https://raw.githubusercontent.com/FelipeGomide/TP2_IBD/main/data/20230424-tabelapandados.csv', encoding='ISO-8859-1', sep=';')
esp_csv = pd.read_csv('https://raw.githubusercontent.com/FelipeGomide/TP2_IBD/main/data/20230424-tabelapanespecies.csv', encoding='unicode_escape', sep=';')
bio_csv = pd.read_csv('https://raw.githubusercontent.com/FelipeGomide/TP2_IBD/main/data/20230424-tabelapanbiomas.csv', encoding='ISO-8859-1', sep=';')
est_csv = pd.read_csv('https://raw.githubusercontent.com/FelipeGomide/TP2_IBD/main/data/20230424-tabelapanestados.csv', encoding='ISO-8859-1', sep=';')

## Tabelas derivadas de pan_csv

### Tabela Projeto

In [ ]:
df = pan_csv

df = df.iloc[:, [2,3,8,17,18]]

df = df.rename(
    columns={
        "panNomeCompleto": "NomeCompleto",
        "panLogo": "Logo",
        "panSite": "Site",
        "panNomeFantasia": "Fantasia",
        "panCentro": "Centro",
    }
)

#Existe repetição, quero manter objetos com menos nulos
df = df.assign(counts=df.count(axis=1)).sort_values(['Fantasia', 'counts']) #Crio uma coluna com o número de atributos na linha, e ordeno pelo nome + quantidade
df = df.drop_duplicates(subset=['Fantasia'], keep='last') #Removo duplicatas do nome, mantendo o ultimo valor (mais atributos)
df = df.drop('counts', axis=1) #Removo a coluna auxiliar

df_projeto = df
df_projeto

,NomeCompleto,Fantasia,Centro,Logo,Site
30,Plano de Ação Nacional para a Conservação dos ...,Albatrozes e Petréis,CEMAVE,https://www.gov.br/icmbio/pt-br/assuntos/biodi...,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
70,Plano de Ação Nacional para a Conservação das ...,Alto Paraná,CEPTA,NaN,NaN
19,Plano de Ação Nacional para Conservação da Ara...,Arara-azul-de-Lear,CEMAVE,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
31,Plano de Ação Nacional para a Conservação da A...,Ararinha-azul,CEMAVE,https://www.gov.br/icmbio/pt-br/assuntos/biodi...,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
53,Plano de Ação Nacional para a Conservação da A...,Ariranha,CENAP,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
...,...,...,...,...,...
104,Plano de Ação Nacional para a Conservação das ...,Tartarugas Marinhas,TAMAR,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
85,Plano de Ação Nacional para Conservação do Tat...,Tatu-bola,CPB,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
78,Plano de Ação Nacional para Conservação da Ton...,Toninha,CMA,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...
60,Plano de Ação Nacional para a Conservação dos ...,Tubarões,CEPSUL,NaN,https://www.gov.br/icmbio/pt-br/assuntos/biodi...


### Tabela Ciclo

In [ ]:
df = pan_csv

df = df.iloc[:, [2,0,6,4,5,7,9,13,15,16]]

df = df.rename(
    columns={
        "panNomeCompleto": "NomeCompleto",
        "idPan": "IDCiclo",
        "panCiclo": 'Ciclo',
        "panAbrangenciaTaxonomica": "AbrangenciaTaxonomica",
        "panAbrangenciaGeografica": "AbrangenciaGeografica",
        "panStatus": "Status",
        "panInicioData": "DataInicio",
        "panFimData": "DataFim",
        "panPortaria": "Portaria",
        "Data da portaria vigente do PAN": "DataPortaria",
        "Data do início da vigência do PAN na portaria \n(último artigo da portaria)": "VigênciaPortaria",
        "panProcesso": "NumeroProcesso",
    }
)

df_ciclo = df
df_ciclo

,NomeCompleto,IDCiclo,Ciclo,AbrangenciaTaxonomica,AbrangenciaGeografica,Status,DataInicio,DataFim,Portaria,NumeroProcesso
0,Plano de Ação Nacional para a Conservação de L...,CBC_01.01,1º ciclo,Intraclasse,Nacional,Finalizado,27/08/2010,01/12/2015,Portaria 92 (27/08/2010),02070.002646/2010-11
1,Plano de Ação Nacional para a Conservação Plan...,CBC_02.01,1º ciclo,Intraclasse,Nacional,Em execução,05/12/2022,31/12/2027,Portaira 1145 (05/12/2022),02176.000061/2019-17
2,Plano de Ação Nacional para Conservação do Mor...,CECAV_01.01,1º ciclo,Monoespecífico,Bioma,Finalizado,27/08/2010,01/09/2015,Portaria 89 (27/08/2010),02070.003685/2009-94
3,Plano de Ação Nacional para a Conservação do P...,CECAV_02.01,1º ciclo,Multiclasse,Ecossistema,Finalizado,17/02/2012,01/02/2017,Portaria 18 (17/02/2012),02070.002654/2010-50
4,Plano de Ação Nacional para a Conservação do P...,CECAV_03.01,1º ciclo,Multiclasse,Ecossistema,Em execução,09/08/2022,01/08/2027,Portaria 646 (09/08/2022),02667.000076/2019-45
...,...,...,...,...,...,...,...,...,...,...
100,Plano de Ação Nacional para a Conservação da H...,RAN_10.02,2º ciclo,Multiclasse,Bioma,Em execução,28/12/2018,01/01/2024,Portaria 354 (25/07/2019),02071.000134/2018-50
101,Plano de Ação Nacional para Conservação da Her...,RAN_11.02,2º ciclo,Multiclasse,Bioma,Previsto,Previsto,NaN,NaN,NaN
102,Plano de Ação Nacional para Conservação das Es...,RAN_12.02,2º ciclo,Multiclasse,Bioma,Previsto,Previsto,NaN,NaN,NaN
103,Plano de Ação Nacional para a Conservação das ...,TAMAR_01.01,1º ciclo,Intraclasse,Bioma,Finalizado,23/12/2010,01/12/2015,Portaria 135 (23/12/2010),NaN


## Tabelas derivadas de esp_csv


### Tabela Espécies

In [ ]:
df = esp_csv

df = df.rename(
    columns={
        "idTaxon": "IDTaxon",
        "taxonNome": "NomeCientifico",
    }
)

df = df.iloc[:, [0,1]]

df = df[df.NomeCientifico != 'não é mais subespécie'] #Removo a linha 315, que tinha uma espécie apagada

df = df.replace({'Crypturellus noctivagus noctivagus Crypturellus noctivagus':'Crypturellus noctivagus noctivagus'}) #Corrijo os nomes de 534 e 856

df[['NomeGenero', 'EpitetoEspecifico', 'Subespecie']] = df['NomeCientifico'].str.split(expand=True) #Quebro a coluna nome científico em 3 nomes

df = df.drop_duplicates()

df = df.drop(columns=['Subespecie'])

df_especies = df
df_especies

#Instâncias com problemas:
# 315, "não é mais subespécie" : nome apagado
# 534 e 856, "Crypturellus noctivagus noctivagus Crypturellus noctivagus" : nome repetido

,IDTaxon,NomeCientifico,NomeGenero,EpitetoEspecifico
0,10488,Cyanopsitta spixii,Cyanopsitta,spixii
1,7915,Adelophryne maranguapensis,Adelophryne,maranguapensis
2,7935,Hylomantis granulosa,Hylomantis,granulosa
3,12115,Glaucomastix abaetensis,Glaucomastix,abaetensis
4,12121,Ameivula nativo,Ameivula,nativo
...,...,...,...,...
2388,3678,Schizodon altoparanae,Schizodon,altoparanae
2389,7341,Zungaro jahu,Zungaro,jahu
2391,6333,Bunocephalus larai,Bunocephalus,larai
2393,7368,Pseudopimelodus mangurus,Pseudopimelodus,mangurus


### Tabela da relação Ciclo-Espécies

In [ ]:
df = esp_csv

df = df.iloc[:, [2,0]]

df = df.rename(
    columns={
        "idTaxon": "IDTaxon",
        "idPan": "IDCiclo",
    }
)

df = df.drop_duplicates()

df_cic_esp = df
df_cic_esp

,IDCiclo,IDTaxon
0,CEMAVE_27.02,10488
1,RAN_10.02,7915
2,RAN_10.02,7935
3,RAN_10.02,12115
4,RAN_10.02,12121
...,...,...
2391,CEPTA_09.01,6333
2392,CEPTA_09.01,4391
2393,CEPTA_09.01,7368
2394,CEPTA_09.01,7191


## Tabelas Derivadas de est_csv

### Tabela Estado

In [ ]:
df = est_csv

df = df.iloc[:, [0,2]]

df = df.rename(
    columns = {
    'idPan': 'IDCiclo',
    'siglaEstado': 'Sigla',
    }
)

df = df.drop(index= 470) #Removo caso de nome de apenas um espaço

df_est = df
df_est

,IDCiclo,Sigla
0,CEMAVE_29.02,MG
1,CEMAVE_29.02,TO
2,CEMAVE_29.02,GO
3,CEMAVE_26.03,PR
4,CEMAVE_26.03,RJ
...,...,...
515,CECAV_03.01,RR
516,CECAV_03.01,SC
517,CECAV_03.01,SP
518,CECAV_03.01,SE


### Tabela de regiões

In [ ]:
df = est_csv

df = df.iloc[:, [2]]

df = df.rename(columns={'siglaEstado':'Sigla'})

df = df.drop(index= 470) #Removo caso de nome de apenas um espaço

df = df.drop_duplicates()

regioes = {
    'AC':'Norte',
    'AL':'Nordeste',
    'AP':'Norte',
    'AM':'Norte',
    'BA':'Nordeste',
    'CE' : 'Nordeste',
    'DF' : 'Centro-Oeste',
    'ES' : 'Sudeste',
    'GO' : 'Centro-Oeste',
    'MA' : 'Nordeste',
    'MT' : 'Centro-Oeste',
    'MS' : 'Centro-Oeste',
    'MG' : 'Sudeste',
    'PA' : 'Norte',
    'PB' : 'Nordeste',
    'PR' : 'Sul',
    'PE' : 'Nordeste',
    'PI' : 'Nordeste',
    'RJ' : 'Sudeste',
    'RN' : 'Nordeste',
    'RS' : 'Sul',
    'RO' : 'Norte',
    'RR' : 'Norte',
    'SC' : 'Sul',
    'SP' : 'Sudeste',
    'SE' : 'Nordeste',
    'TO' : 'Norte'
 }

df['Regiao'] = df['Sigla'].map(regioes)

df_regiao = df
df_regiao

,Sigla,Regiao
0,MG,Sudeste
1,TO,Norte
2,GO,Centro-Oeste
3,PR,Sul
4,RJ,Sudeste
5,RS,Sul
6,SC,Sul
7,SP,Sudeste
8,ES,Sudeste
9,BA,Nordeste


##Tabelas derivadas de bio_csv


### Tabela Bioma

In [ ]:
df = bio_csv

df = df.iloc[:, [0,2]]

df = df.rename(
    columns={
        "idPan": "IDCiclo",
        "panBioma": "NomeBioma"
    }
)

df = df.drop_duplicates()

df_bioma = df
df_bioma

,IDCiclo,NomeBioma
0,CBC_01.01,Amazônia
1,CBC_01.01,Caatinga
2,CBC_01.01,Cerrado
3,CBC_01.01,Mata Atlântica
4,CBC_01.01,Pampa
...,...,...
221,RAN_09.02,Pampa
222,RAN_10.02,Mata Atlântica
223,RAN_10.02,Caatinga
224,TAMAR_01.01,Marinho


## Tabela Localização

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/FelipeGomide/TP2_IBD/main/data/20230424-tabelapanestados.csv', encoding='ISO-8859-1', sep=';')

df = df.iloc[:, [0,2]]

df = df.rename(
    columns={
        'idPan': 'IDPan',
        'siglaEstado': 'Estado'
    }
)

#Obrigado chatGPT, preciso conferir isso direito
biomas = {
    'AC': ['Floresta Amazônica'],
    'AL': ['Mata Atlântica', 'Caatinga', 'Marinho'],
    'AM': ['Floresta Amazônica'],
    'AP': ['Floresta Amazônica','Marinho'],
    'BA': ['Mata Atlântica', 'Caatinga', 'Marinho','Cerrado'],
    'CE': ['Caatinga', 'Marinho'],
    'DF': ['Cerrado'],
    'ES': ['Mata Atlântica', 'Marinho'],
    'GO': ['Cerrado','Mata Atlântica'], #Polêmico
    'MA': ['Floresta Amazônica', 'Marinho','Cerrado'],
    'MG': ['Mata Atlântica', 'Cerrado','Caatinga'], #Polêmico
    'MS': ['Cerrado', 'Pantanal','Mata Atlântica'],
    'MT': ['Cerrado', 'Floresta Amazônica','Pantanal'],
    'PA': ['Floresta Amazônica', 'Marinho'],
    'PB': ['Mata Atlântica', 'Caatinga', 'Marinho'],
    'PE': ['Mata Atlântica', 'Caatinga', 'Marinho'],
    'PI': ['Mata Atlântica', 'Caatinga', 'Marinho','Cerrado'],
    'PR': ['Mata Atlântica','Marinho'],
    'RJ': ['Mata Atlântica', 'Marinho'],
    'RN': ['Mata Atlântica', 'Caatinga', 'Marinho'],
    'RO': ['Floresta Amazônica'],
    'RR': ['Floresta Amazônica'],
    'RS': ['Pampas','Mata Atlântica'],
    'SC': ['Mata Atlântica'],
    'SE': ['Mata Atlântica', 'Caatinga', 'Marinho'],
    'SP': ['Mata Atlântica', 'Cerrado', 'Marinho'],
    'TO': ['Floresta Amazônica', 'Cerrado'],
}

list_test = []

for key, value in biomas.items():
  for state in value:
      list_test.append({'Sigla':key,'NomeBioma':state})

df_localizacao = pd.DataFrame(list_test)

df_localizacao

,Sigla,NomeBioma
0,AC,Floresta Amazônica
1,AL,Mata Atlântica
2,AL,Caatinga
3,AL,Marinho
4,AM,Floresta Amazônica
...,...,...
57,SP,Mata Atlântica
58,SP,Cerrado
59,SP,Marinho
60,TO,Floresta Amazônica


# Criando o Banco de Dados

In [ ]:
import sqlite3

conn = sqlite3.connect("data_tp2.sql")
cur = conn.cursor()

cur.execute("CREATE TABLE IF NOT EXISTS Projeto(NomeCompleto VARCHAR(50), Logo VARCHAR(50), Site VARCHAR(60), Fantasia VARCHAR(30), Centro VARCHAR(10), PRIMARY KEY (NomeCompleto));")

cur.execute("CREATE TABLE IF NOT EXISTS Ciclo(NomeCompleto VARCHAR(50), IDCiclo INTERGER, Ciclo VARCHAR(10), DataInicio DATE, DataFim DATE, Portaria VARCHAR(30), NumeroProcesso VARCHAR(30), AbrangenciaGeografica VARCHAR(10), AbrangenciaTaxonomica VARCHAR(10), Status VARCHAR(15), PRIMARY KEY (NomeCompleto, IDCiclo), FOREIGN KEY (NomeCompleto) REFERENCES Projeto(NomeCompleto));")

cur.execute("CREATE TABLE IF NOT EXISTS Especies(IDTaxon INTERGER, NomeCientifico VARCHAR(50), NomeGenero VARCHAR(20), EpitetoEspecifico VARCHAR (20), PRIMARY KEY (IDTaxon));")

cur.execute("CREATE TABLE IF NOT EXISTS CicloEspecies(IDTaxon INTERGER, IDCiclo INTERGER, PRIMARY KEY (IDTaxon, IDCiclo), FOREIGN KEY (IDTaxon) REFERENCES Especies(IDTaxon), FOREIGN KEY (IDCiclo) REFERENCES Ciclo(IDCiclo));")

cur.execute("CREATE TABLE IF NOT EXISTS Bioma(IDCiclo INTERGER, NomeBioma VARCHAR(15), PRIMARY KEY (IDCiclo, NomeBioma), FOREIGN KEY (IDCiclo) REFERENCES Ciclo(IDCiclo));")

cur.execute("CREATE TABLE IF NOT EXISTS Estado(IDCiclo INTERGER, Sigla VARCHAR(2), PRIMARY KEY (IDCiclo, Sigla), FOREIGN KEY (IDCiclo) REFERENCES Ciclo(IDCiclo));")

cur.execute("CREATE TABLE IF NOT EXISTS Localizacao(NomeBioma VARCHAR(15), Sigla VARCHAR(2), PRIMARY KEY (NomeBioma, Sigla), FOREIGN KEY (Sigla) REFERENCES Estado(Sigla), FOREIGN KEY (NomeBioma) REFERENCES Bioma(NomeBioma));")

cur.execute("CREATE TABLE IF NOT EXISTS Regiao(Sigla VARCHAR(2), Regiao VARCHAR(15), PRIMARY KEY (Sigla), FOREIGN KEY (Sigla) REFERENCES Estado(Sigla));")

conn.commit()



## Preenchimento das tabelas

### Preenche as tabelas

In [ ]:
df_projeto.to_sql('Projeto', conn, if_exists='append', index = False)

df_ciclo.to_sql('Ciclo', conn, if_exists='append', index = False)

df_especies.to_sql('Especies', conn, if_exists='append', index = False)

df_cic_esp.to_sql('CicloEspecies', conn, if_exists='append', index = False)

df_est.to_sql('Estado', conn, if_exists='append', index = False)

df_regiao.to_sql('Regiao', conn, if_exists='append', index = False)

df_bioma.to_sql('Bioma', conn, if_exists='append', index = False)

df_localizacao.to_sql('Localizacao', conn, if_exists='append', index = False)

62

### Criando Dump

In [ ]:
with open('dump.sql', 'w') as f:
    for line in conn.iterdump():
        f.write('%s\n' % line)

In [ ]:
conn.close()
!rm -rf /content/data_tp2.sql